## 기업 개황(Overview): 여러 회사
+ 작성: 임경호
+ 금융감독원 OPEN DART https://opendart.fss.or.kr/

In [45]:
import pandas as pd

import sys
module_path = "D:\PythonProject\data-gatherer\dart_fs_notes"
sys.path.append(module_path)
import myutil
 
# 파일 저장 위치
path_dir = "D:/PythonProject/data-gatherer/dart_fs_notes/company/"

In [46]:
# 기업코드 목록 전체
file_name = "corp_codes_all.csv"
df_corp = pd.read_csv(path_dir + file_name, dtype=object)
list_all_codes = df_corp['corp_code'].unique()
len(list_all_codes)

103859

In [47]:
# 기업개황 저장된 목록 전체
file_name = "corp_info_all.csv"
df_own_corp_info = pd.read_csv(path_dir + file_name, dtype=object)
list_own_codes = df_own_corp_info['corp_code'].unique()
len(list_own_codes)

103859

In [48]:
# 리스트에서 중복 제거 (이미 corp info가 있는 경우 대상에서 제외)
list_target_codes = list(set(list_all_codes) - set(list_own_codes))
len(list_target_codes)

0

* corp_code 분할

In [49]:
list_cnt = len(list_target_codes)
n = 1000
list_of_lists = [list_target_codes[i * n:(i + 1) * n] for i in range((list_cnt + n - 1) // n )] 
len(list_of_lists)

0

* 기업 개황 정보 가져오기

In [33]:
from tqdm import tqdm
from time import sleep
import requests

# Company Info   
url = 'https://opendart.fss.or.kr/api/company.json'
api_key = 'f2e08d4ed3de0ba3d5cbf59c04c223e02b1751a2'

for corp_codes in list_of_lists:
    corp_list = []
    for corp_code in tqdm(corp_codes):
        sleep(0.1)
        params = {
            'corp_code': corp_code,
            'crtfc_key': api_key,
        }
        try:
            response = requests.get(url, params=params)     
            if response.status_code == 200:     # URL GET '200 정상'
                json_data = response.json()
                if json_data['status'] == '020':    # 사용한도 초과
                    print(json_data['message'])
                    raise Exception
                else:
                    corp_list.append(json_data)
            else:
                print("URL GET Error", corp_code)
                break
        except Exception as e:
            print(e)
            break
    if len(corp_list) == 0:
        break
    else:
        # 데이터프레임 형태로 변환
        df_save_corp = pd.DataFrame(corp_list)
        # csv 파일로 저장
        df_save_corp['induty_code'] = df_save_corp['induty_code'].str.zfill(5)  # 산업분류코드 5자리(문자열 앞에 0 채우기)
        df_save_corp = df_save_corp.astype({'corp_code' : 'string', 'stock_code' : 'string', 'induty_code' : 'string'})
        file_path = path_dir + 'corp_info_' + now_dt_str() + '.csv'
        df_save_corp.to_csv(file_path, index=False)

100%|██████████| 569/569 [02:38<00:00,  3.60it/s]


In [50]:
# 목록 비교
df_left_only = pd.merge(df_corp, df_own_corp_info, how='outer', left_on='corp_code', right_on='corp_code' , indicator=True).query("_merge=='left_only'")
df_right_only = pd.merge(df_corp, df_own_corp_info, how='outer', left_on='corp_code', right_on='corp_code' , indicator=True).query("_merge=='right_only'")

In [51]:
df_left_only

,corp_code,corp_name_x,stock_code_x,modify_date,status,message,corp_name_y,corp_name_eng,stock_name,stock_code_y,...,bizr_no,adres,hm_url,ir_url,phn_no,fax_no,induty_code,est_dt,acc_mt,_merge


In [52]:
df_right_only

,corp_code,corp_name_x,stock_code_x,modify_date,status,message,corp_name_y,corp_name_eng,stock_name,stock_code_y,...,bizr_no,adres,hm_url,ir_url,phn_no,fax_no,induty_code,est_dt,acc_mt,_merge
